In [1]:
import cv2
import numpy as np
import tensorflow as tf
import mediapipe as mp
import pickle

# Load your trained MobileNet model
mobilenet_model = tf.keras.models.load_model(
    r"C:\Users\moham\Downloads\Robotech\Summer_Training\Project\Note_Book\Small_Data_model\MediaPipe\Small_Model_Final2.h5"
)

# Load label dictionary
with open(r"C:\Users\moham\Downloads\Robotech\Summer_Training\Project\Note_Book\Small_Data_model\MediaPipe\label_dict.pkl", "rb") as f:
    label_dict = pickle.load(f)

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(
    static_image_mode=False,     
    max_num_hands=1,               
    min_detection_confidence=0.6,
    min_tracking_confidence=0.6
)

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = hands.process(rgb_frame)

    if result.multi_hand_landmarks:
        for hand_landmarks in result.multi_hand_landmarks:

            h, w, c = frame.shape
            x_coords = [int(lm.x * w) for lm in hand_landmarks.landmark]
            y_coords = [int(lm.y * h) for lm in hand_landmarks.landmark]

            x1, x2 = min(x_coords), max(x_coords)
            y1, y2 = min(y_coords), max(y_coords)

            padding = 20
            x1, y1 = max(0, x1 - padding), max(0, y1 - padding)
            x2, y2 = min(w, x2 + padding), min(h, y2 + padding)

            hand_img = frame[y1:y2, x1:x2]

            if hand_img.size > 0:
                hand_resized = cv2.resize(hand_img, (224, 224))
                hand_resized = hand_resized.astype("float32") / 255.0
                hand_resized = np.expand_dims(hand_resized, axis=0)

                preds = mobilenet_model.predict(hand_resized, verbose=0)
                class_id = int(np.argmax(preds))
                confidence = float(np.max(preds))

                if confidence > 0.6:
                    letter = label_dict[class_id]
                    label = f"{letter}: {confidence:.2f}"

                    cv2.putText(frame, label, (x1, y1 - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

    cv2.imshow("Sign Language Detection (Mediapipe)", frame)

    if cv2.waitKey(1) & 0xFF == 27:  # ESC to exit
        break

cap.release()
cv2.destroyAllWindows()
